# EEE466 Lab 4: Distributed Transactions

## Part 1: Background

The objective of this lab is for you to design and implement a client-server architecture for handling distributed transactions. The distributed transaction model involves a Coordinator and two or more Participants that will vote to either commit or abort a given transaction. A popular algorithm used to this voting process is the Two-Phase Commit (2PC). The coordinator and participant entities are decoupled from the client and server. It is possible that server calls a different process or a different machine to act as the coordinator, or a client runs the coordinator and other servers are participants.

## Part 2: Design

In this lab, you must design a server that will handle transaction requests and implement transactions by interacting with at least 2 clients. Clients come in two types: buyers and sellers. In order for a transaction to take place, there must be a request from at least one of each client type to the server. Once the server has determined that a transaction is required, it will respond to the requests and create a coordinator object. The clients will each create a participant object. The interaction at this point has moved from client-server to coordinator-participant. To emphasize the decoupling of these models, sockets must not be shared between server and coordinator or between client and participant. Assume that the server and clients have other work to do in the meantime. During the initial handshake server will send the clients a randomly generated port number that they pass to their respective participants to create new socket connections.

For example, a server is hosting requests to port 443. Upon receiving a request from a client, it will respond with a port number (4321). This port will be used by the coordinator as the destination for 2PC messages. The client then creates a participant that listens on port 4321 for a vote request from the coordinator, starting the 2PC algorithm.

`auction.json` contains the *market value* of a series of rare items sold at auction. All clients can read this file. The buyer client is looking to get a good deal and will buy anything as long as the price is **at or below market value**. The seller will list an item (via server request) with a randomly generated price that is close to market value +/- 10%. If it meets the buyer's criteria, the buyer will vote to commit the transaction, otherwise it will abort. The coordinator must implement 2PC so that the buyer and seller know if their transaction has been successful or not.

Figure 1 shows a simplified sequence diagram of Client and Server actions. Note that these actions occur asynchronously and in any order. The server hosts a listening socket for new buyer and seller requests continuously. The Coordinator and Participants are created and destroyed with new ports for every new item.

![Figure 1: Simplified sequence diagram for a single auction item](./lab4_actions.png)


## Part 3: Requirements

### General
- Use the `run.py` file to launch the Server, BuyerClient, and SellerClient as separate threads running concurrently.
- `Participant` and `Coordinator` do not import `Server` or `Client`.
- `time.sleep` must NOT be used. `socket.timeout` can be used. This is to ensure that you design robust concurrency logic. Real-world system response time is unpredictable, e.g., a static 3 second wait for an event is not acceptable.
- Communication between server-client and coordinator-participant must use TCP sockets. Client-client and participant-participant communication is not allowed.

### Server
- `Server` listens for TCP connections.
- `Server` expects one `BuyerClient` and one `SellerClient` to initiate a transaction.
- `Server` instantiates and runs a separate `Coordinator` for each transaction request, i.e., a client selling an item.
- `Server` generates a port numbers (`_generate_port()`) that participants will use to receive 2PC messages.
- `Server` closes TCP connections once it has responded to requests (a single request-reply cycle for each client).
- `Server` does not need to handle multiple transactions concurrently. `Server` does not need to hand multiple request-reply calls to clients concurrently.


### Client
- `BuyerClient` and `SellerClient` inherit from `Client`. `Client` should include the common logic between the buyer and seller clients.
- `SellerClient` selects an item from the provided `auction.json`, prices it using the provided function `_set_price()`, and sends a sell request to `Server`.
- `BuyerClient` sends a request to be informed of the next transaction.
- Both clients, once acknowledged by the server, will close their connections to the server, then instantiate and run their own `Participant` - subclassed into `BuyerParticipant` and `SellerParticipant`.
- Once `SellerClient` has attempted 6 transactions, it will ask `Server` to shutdown. The `Server` must inform `BuyerClient` so that all threads exit gracefully.
- Neither `BuyerClient` nor `SellerClient` need to be informed of the transaction result, this is saved by `Coordinator`

### Participant
- Participants only exist for the duration of a transaction and are then destroyed.
- `Participant` creates a new listening socket using the port number provided by its respective `Client`.
- `Participant` handles 2PC messages with `Coordinator`
- `BuyerParticipant` will only commit if the advertised price for an item is equal to or less than the price in `auction.json`.

### Coordinator
- `Coordinator` only exists for the duration of a transaction and is then destroyed.
- `Coordinator` will log whether each transaction was successful and what the price of the item was. Use the provided function `_save_transaction()` for grading.
- `Coordinator` and `Participant` will close all connections and exit after they have completed 2PC.

Note: this lab requires significant design effort. Take the time to plan how each element will interact before starting. You may use protobuf encoding to handle different request types.

Note: `transaction_logger.py` has been provided to assist you with debugging, [logging](https://docs.python.org/3/library/logging.html) provides a more robust way to handle printing information in concurrent code. Below is an example of using logging track the progress of a transaction.

```
Server:Server listening on ('127.0.0.1', 9001)
Server:Connection from ('127.0.0.1', 52019)
Server:Connection from ('127.0.0.1', 52020)
SellerClient:Listing item: Lunch with Warren Buffett for $4231113 (item 1/6).
Server:Received request to auction item: Lunch with Warren Buffett
BuyerClient:Bidding on item: Lunch with Warren Buffett.
Server:Initiating transaction id 6403 with 1 buyers.
Server:Participant ports for this transaction: [59268, 64818]
2PC:Buyer:59268:Accepted connection from coordinator (port 52021), waiting for vote request.
2PC:Seller:64818:Accepted connection from coordinator (port 52022), waiting for vote request.
2PC:Coordinator:Participant 0 (port 59268) voted to commit.
2PC:Coordinator:Participant 1 (port 64818) voted to commit.
2PC:Coordinator:All participants voted to commit, sending global commit.
2PC:Buyer:59268:Received global commit for transaction 6403, committing.
2PC:Coordinator:Received acknowledgment from participant (port 59268).
2PC:Seller:64818:Received global commit for transaction 6403, committing.
2PC:Coordinator:Received acknowledgment from participant (port 64818).
Coordinator:Saving transaction result: item=Lunch with Warren Buffett, price=4231113, success=True
```

## Part 4: Questions
1. For each property in ACID, describe how your implementation does or does not adhere.
2. What extra logic would need to be applied if the auction included multiple buyer participants?
3. Describe how distributed deadlock could occur if the buy-sell transaction implemented here had dependencies on other transactions.


## Part 5: Submission

Submit a zip archive in the format `eee446_lab4_lastname1_lastname2.zip` that includes the following:
1. A lab report notebook including a *brief* report detailing your design choices during this lab and your answers to the questions.
2. Your project folder including all python and json files.